# Why Weights & Biases Gives You An Edge On Kaggle Competitions

Excited to announce that [Weights & Biases](https://www.wandb.com/) now comes baked into your Kaggle kernels! Used by the likes of [OpenAI](http://openai.com/) and [Github](https://github.blog/2019-09-26-introducing-the-codesearchnet-challenge/), W&B is part of the new standard of best practices for machine learning.

I thought I'd write a quick post on why my fellow Kagglers might find W&B useful, and how you can integrate it into your projects with just a few lines of code. You can find the [full blog post here](https://www.wandb.com/articles/better-models-faster-with-weights-biases) – I highly encourage you check it out to run a more efficient ML process than your peers.

**Tl;dr –** W&B helps you visualize your model performance and predictions, find the best models fast and share insights learned from your models.

Here are a few use cases in which W&B is specially useful for Kagglers:
1. **Track and compare models:** I want to test out my hypotheses fast and iterate quickly to find the best model
2. **Visualize model performance for debugging:** I want to see how my models are performing in real time and debug them
3. **Efficient hyperparameter search:** I want to find the best model faster than everyone else
4. **Resource efficient model training:** I want to be efficient with my model training and not spend more money than I need to
5. **Show off my model:** I want to share my models and key insights with my teammates and the Kaggle community

The blog post goes into detail about how W&B helps address all of these needs. In this Kernel, I want to show you how to integrate W&B in your models in just a few lines of code.

First, here's a quick overview of some of the visualizations W&B automatically creates for your models:

### Visualize model performance and training metrics
<img src="https://paper-attachments.dropbox.com/s_92F7A2BE132D5E4492B0E3FF3430FFF0FB2390A4135C0D77582A2D21A2EF8567_1574108018540_Screenshot+2019-11-18+12.13.29.png" style="width: 80%"/>

### Visualize the affect of hyperparameters on model performance
<img src="https://paper-attachments.dropbox.com/s_C0EC7008D045FC80715C08E7386E0BBDA59DC92DEE34C734FEA67BF25E4BA5CC_1578297697453_image.png" style="width: 80%"/>

### Visualize model predictions
<img src="https://paper-attachments.dropbox.com/s_C0EC7008D045FC80715C08E7386E0BBDA59DC92DEE34C734FEA67BF25E4BA5CC_1578297173261_vzye9ei.png" style="width: 80%"/>

### Visualize gradients to help deal with vanishing and exploding gradients
<img src="https://paper-attachments.dropbox.com/s_C0EC7008D045FC80715C08E7386E0BBDA59DC92DEE34C734FEA67BF25E4BA5CC_1578298883597_image.png" style="width: 80%"/>

### Visualize system metrics, including GPU usage
<img src="https://paper-attachments.dropbox.com/s_C0EC7008D045FC80715C08E7386E0BBDA59DC92DEE34C734FEA67BF25E4BA5CC_1578297329745_image.png" style="width: 80%"/>

### Visualize images, videos, html, audio, 3D objects, plots, tables, point clouds
<img src="https://paper-attachments.dropbox.com/s_C0EC7008D045FC80715C08E7386E0BBDA59DC92DEE34C734FEA67BF25E4BA5CC_1578297638486_image.png" style="width: 80%"/>

As you can see W&B structures your metrics & predictions, and presents them in a way that provides actionable insights about your training process.

Alright, now that you know how W&B can be useful, let's see how you can integrate it into your project and get all these visualizations, with just a few lines of code.

If you have an existing kernel, you just need to switch your Docker image to 'Latest Available'. All new kernels should work out of the box.

# 1. Log any metric with Weights and Biases
*   **wandb.init()** – Initialize a new W&B run. Each run is single execution of the training script.
- **wandb.log()** – Logs custom objects like images, videos, audio files, HTML, plots, point clouds etc.
-   **%%wandb** – Add this at the top of a cell to show model metrics live below the cell

In [ ]:
# Get Apple stock price data from https://www.macrotrends.net/stocks/charts/AAPL/apple/stock-price-history
import pandas as pd
import wandb

# Read in dataset
apple = pd.read_csv("../input/kernel-files/apple.csv")
apple = apple[-1000:]
wandb.init(project="visualize-models", name="a_metric")

In [ ]:
%%wandb
# Log the metric
for price in apple['close']:
    wandb.log({"Stock Price": price})

# 2. Monitor boosting model performance

Start out by importing the experiment tracking library and setting up your free W&B account:

*   **import wandb** – Import the wandb library
*   **callbacks=[wandb.xgboost.wandb_callback()]** – Add the wandb [XGBoost callback](https://docs.wandb.com/library/frameworks/xgboost), or
*   **callbacks=[wandb.lightgbm.wandb_callback()]** – Add the wandb LightGBM callback

In [ ]:
# Get the dataset from UCI
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/dermatology/dermatology.data -qq

# modified from https://github.com/dmlc/xgboost/blob/master/demo/multiclass_classification/train.py
# Import wandb
import wandb
import numpy as np
import xgboost as xgb

wandb.init(project="visualize-models", name="xgboost")

# label need to be 0 to num_class -1
data = np.loadtxt('./dermatology.data', delimiter=',',
        converters={33: lambda x:int(x == '?'), 34: lambda x:int(x) - 1})
sz = data.shape

train = data[:int(sz[0] * 0.7), :]
test = data[int(sz[0] * 0.7):, :]

train_X = train[:, :33]
train_Y = train[:, 34]

test_X = test[:, :33]
test_Y = test[:, 34]

xg_train = xgb.DMatrix(train_X, label=train_Y)
xg_test = xgb.DMatrix(test_X, label=test_Y)
# setup parameters for xgboost
param = {}
# use softmax multi-class classification
param['objective'] = 'multi:softmax'
# scale weight of positive examples
param['eta'] = 0.1
param['max_depth'] = 6
param['silent'] = 1
param['nthread'] = 4
param['num_class'] = 6
wandb.config.update(param)

watchlist = [(xg_train, 'train'), (xg_test, 'test')]
num_round = 5

In [ ]:
%%wandb
# Add the wandb xgboost callback
bst = xgb.train(param, xg_train, num_round, watchlist, callbacks=[wandb.xgboost.wandb_callback()])
# get prediction
pred = bst.predict(xg_test)
error_rate = np.sum(pred != test_Y) / test_Y.shape[0]
print('Test error using softmax = {}'.format(error_rate))

wandb.summary['Error Rate'] = error_rate

# 3. Monitor scikit learn performance

Logging sklearn plots with Weights & Biases is simple.

### Step 1: First import wandb and initialize a new run.
```
import wandb
wandb.init(project="visualize-sklearn")

# load and preprocess dataset
# train a model
```

### Step 2: Visualize individual plots.
```
# Visualize single plot
wandb.sklearn.plot_confusion_matrix(y_true, y_probas, labels)
```

### Or visualize all plots at once:
```python
# Visualize all classifier plots
wandb.sklearn.plot_classifier(clf, X_train, X_test, y_train, y_test, y_pred, y_probas, labels, model_name='SVC', feature_names=None)

# All regression plots
wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test,  model_name='Ridge')

# All clustering plots
wandb.sklearn.plot_clusterer(kmeans, X_train, cluster_labels, labels=None, model_name='KMeans')
```

In [ ]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
import matplotlib.pyplot as plt
import pandas as pd
import wandb
# wandb.init(anonymous='allow', project="sklearn")
wandb.init(project="visualize-models", name="sklearn")

# Load data
boston = load_boston()
X = pd.DataFrame(boston.data, columns=boston.feature_names)
y = boston.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Train model, get predictions
reg = Ridge()
reg.fit(X, y)
y_pred = reg.predict(X_test)

# Visualize model performance
wandb.sklearn.plot_regressor(reg, X_train, X_test, y_train, y_test, 'Ridge')

# 4. Monitor neural network performance

Start out by installing the experiment tracking library and setting up your free W&B account:

*   **import wandb** – Import the wandb library
*   **from wandb.keras import WandbCallback** – Import the wandb [keras callback](https://docs.wandb.com/library/frameworks/keras)

In [ ]:
# WandB – Import the W&B library
import wandb
from wandb.keras import WandbCallback

from keras.datasets import fashion_mnist
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Dense, Flatten
from keras.utils import np_utils
from keras.optimizers import SGD
from keras.callbacks import TensorBoard

*   **wandb.init()** – Initialize a new W&B run. Each run is single execution of the training script.
*   **wandb.config** – Save all your hyperparameters in a config object. This lets you use W&B app to sort and compare your runs by hyperparameter values.
*   **callbacks=[WandbCallback()]** – Fetch all layer dimensions, model parameters and log them automatically to your W&B dashboard.

In [ ]:
# Default values for hyper-parameters
defaults=dict(
    dropout = 0.2,
    hidden_layer_size = 32,
    layer_1_size = 32,
    learn_rate = 0.01,
    decay = 1e-6,
    momentum = 0.9,
    epochs = 5,
    )

# Initialize a new wandb run and pass in the config object
# wandb.init(anonymous='allow', project="kaggle", config=defaults)
wandb.init(project="visualize-models", config=defaults, name="neural_network")
config = wandb.config

(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
labels=["T-shirt/top","Trouser","Pullover","Dress","Coat",
        "Sandal","Shirt","Sneaker","Bag","Ankle boot"]

img_width=28
img_height=28

X_train = X_train.astype('float32')
X_train /= 255.
X_test = X_test.astype('float32')
X_test /= 255.

#reshape input data
X_train = X_train.reshape(X_train.shape[0], img_width, img_height, 1)[:10000]
X_test = X_test.reshape(X_test.shape[0], img_width, img_height, 1)[:10000]

# one hot encode outputs
y_train = np_utils.to_categorical(y_train)[:10000]
y_test = np_utils.to_categorical(y_test)[:10000]
num_classes = y_test.shape[1]

# build model
model = Sequential()
model.add(Conv2D(config.layer_1_size, (5, 5), activation='relu',
                            input_shape=(img_width, img_height,1)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(config.dropout))
model.add(Flatten())
model.add(Dense(num_classes, activation='softmax'))

sgd = SGD(lr=config.learn_rate, decay=config.decay, momentum=config.momentum, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

In [ ]:
%%wandb
# Add WandbCallback() to the fit function
model.fit(X_train, y_train,  validation_data=(X_test, y_test), epochs=config.epochs,
    callbacks=[WandbCallback(data_type="image", labels=labels)])

## Find the best neural network model with hyperparameter sweeps

Searching through high dimensional hyperparameter spaces to find the winning model can get unwieldy very fast. W&B's hyperparameter sweeps provide an organized and efficient way to conduct a battle royale of models and pick the best one.

Running a hyperparameter sweep with Weights & Biases is very easy. You can use [this guide](https://docs.wandb.com/sweeps/overview/getting-started) to get started.

![](https://paper-attachments.dropbox.com/s_A8A9577ACEF2EF9A66A68CAA0D798FE3970C9A78CA8BF44A10FA307611490E90_1572034183402_Screenshot+2019-10-25+13.09.37.png)

## More Resources

Check out some other cool things you can do with Weights & Biases:
* [Visualize sklearn models](https://app.wandb.ai/lavanyashukla/visualize-sklearn/reports/Visualize-Sklearn-Model-Performance--Vmlldzo0ODIzNg)
* [Visualize model predictions](https://app.wandb.ai/lavanyashukla/visualize-predictions/reports/Visualize-Model-Predictions--Vmlldzo1NjM4OA/)
* [Track model performance](https://app.wandb.ai/lavanyashukla/visualize-models/reports/Visualize-Model-Performance--Vmlldzo1NTk2MA)